# Noise Exercise

This notebook contains an exercise divided into six (very interconnected) parts. 
For each part, you need to write a bit of code, and the cells towards the end of the notebook contain calls that generate images. Use this to check whether your implementations work.

When you have completed all parts, please answer the questions at the end of this notebook. Your answers should just be added to the notebook, and the entire notebook is submitted as a hand-in by exporting it as HTML and then uploading the HTML to the assignment in Learn.

### Making this exercise easier

Note that Part 4 is non-mandatory.

### Making this exercise harder

To make this exercise more challenging consider one of the following _non-mandatory_ extensions.

- implement one or more different noise functions, e.g. Perlin's noise function:   
  https://findit.dtu.dk/en/catalog/5a2ff7225010df33b0055c9c 
  or sparse convolution noise:  
  https://backend.orbit.dtu.dk/ws/portalfiles/portal/4164648/imm5481.pdf
- implement Bridson's algorithm for fast Poisson Disk Sampling:  
  https://www.cs.ubc.ca/~rbridson/docs/bridson-siggraph07-poissondisk.pdf

#### Implementation details
The libraries needed are probably part of your distribution, but you will need to install PyGEL. For most users, the best way to do that is via `pip`:

```pip install --user pygel3d```

Chances are that `numpy` and `numba` are already installed, but, otherwise, you can also get those using `pip`. `plotly`is installed as a dependency of `pygel3d`. Unfortunately, the very latest version of plotly has issues regarding HTML export. Please explicitly install `plotly` using 

```pip install --user plotly==5.24.1```

## Initialization in the block below

In [ ]:
from math import cos, sin, pi, sqrt, exp, atan2
from numpy import ndindex, array, ndarray, dot, empty, linspace, clip, abs, ceil, zeros, repeat
from numpy.linalg import norm
from numpy.random import normal
from numpy.fft import fft2,fftshift
from random import uniform
from numba import njit
from time import time
import plotly.express as px
from pygel3d import hmesh, jupyter_display as jd
jd.set_export_mode(True)

@njit
def vec2(x,y):
    '''Convert two numbers to a 2D numpy array '''
    v = empty((2,))
    v[0] = x
    v[1] = y
    return v

@njit
def vec3(x,y,z):
    '''Convert three numbers to a 2D numpy array '''
    v = empty((3,))
    v[0] = x
    v[1] = y
    v[2] = z
    return v

@njit
def vec4(x,y,z,w):
    '''Convert four numbers to a 2D numpy array '''
    v = empty((4,))
    v[0] = x
    v[1] = y
    v[2] = z
    v[3] = w
    return v

def Fourier_transform(img):
    ft = fft2(img)
    ft = fftshift(ft)
    ft_mag = clip(abs(ft),0,256)
    return ft_mag

## Insert code in blocks below

In [2]:
@njit
def init_noise_vectors(N):
    '''Initialize random vectors used to generate the 2D sine waves that
    form the basis of the noise. The parameter N is the number of sine 
    waves used. The function returns an Nx4 matrix each row of which 
    retpresents a wave contributing to the noise.'''
    noise_waves = empty((N,4))
    #--- Part 1: Insert code below that initializes noise_waves
    sigma = 10
    for i in range(N):
        ksi = normal(0, sigma)
        omega_x = (3* sigma + ksi) * cos(2*pi*i/N)
        omega_y = (3* sigma + ksi) * sin(2*pi*i/N)
        phi = uniform(0, 2*pi)
        weight = exp((-ksi**2)/(sigma**2))
        noise_waves[i] = vec4(omega_x, omega_y, phi, weight)
    #--- End of Part 1
    return noise_waves

WAVES = init_noise_vectors(90)


In [14]:
@njit
def noise(x):
    '''Noise function. Simply sums a number of sine functions'''
    I = 0.0
    #--- Part 2: Insert code below that sums waves. Store sum in I
    for i in range(WAVES.shape[0]):
        omega_x = WAVES[i,0]
        omega_y = WAVES[i,1]
        phi = WAVES[i,2]
        weight = WAVES[i,3]
        I += weight * sin(omega_x * x[0] + omega_y * x[1] + phi)
    #--- End of Part 2
    return I


In [4]:
@njit
def turbulence(x,octaves=10):
    '''Add octaves of noise to produce a turbulence-like result. Returns 
    a single number which is the intensity of the turbulence.'''
    I = 0
    #--- Part 3: Insert code that sums octaves of noise producing "turbulence"

    #--- End of Part 3
    return I


In [5]:
@njit
def periodic_fun(t, scale):
    '''This function combines noise and periodic functions to produce an interesting image/texture.'''
    I = 0
    #--- Part 4: go crazy (non-mandatory)

    #--- End of Part 4
    return I



In [6]:
@njit
def noise_grad(x):
    '''Gradient of noise function computed analytically'''
    g = vec2(0,0)
    #--- Part 5: Sum gradients of each wave contributing to noise

    #--- End of Part 5
    return g


In [7]:
@njit
def Poisson_Disk_Sampling():
    '''Create a distribution of N points such that no two points are 
    closer than R.'''
    N = 100
    R = 0.085	
    pts = empty((N,2))
    #--- Part 6: Fill the array with points abiding by ||pts[i]-pts[j]||>=R

    #--- End of Part 6
    return pts, R

## Helper functions below

In [8]:
@njit
def V(x, noise_scale):
    ''' V returns a curl noise vector field. In 2D this is just
    a 90 degree turn of the gradient of the noise.'''
    p = noise_scale * x
    g = noise_grad(p)
    gh = vec2(-g[1], g[0])
    return gh

@njit
def Rand(x, dt, noise_scale):
    '''Perform a random step from x in a random direction.'''
    return x + dt * 0.5*vec2(uniform(-0.5,0.5),uniform(-0.5,0.5))

@njit
def RK4(x, dt, noise_scale):
    '''Perform one Runge-Kutta 4th order step of length dt from x along the curl noise vector field'''
    a = dt * V(x, noise_scale)
    b = dt * V(x+a/2, noise_scale)
    c = dt * V(x+b/2, noise_scale)
    d = dt * V(x+c, noise_scale)
    return x + (a+2*b+2*c+d)/6


def show_function_and_spectrum(fun, scale):
    '''Display image of function together with Fourier spectrum'''
    dim = 1024
    img = empty((dim,dim))
    s = (scale/dim)
    for i,j in ndindex((dim,dim)):
        p = s*vec2(i,j)
        img[i,j] = fun(p)
    fig1 = px.imshow(img, width=800, height=800)
    fig2 = px.imshow(Fourier_transform(img), width=800, height=800)
    fig1.show()
    fig2.show()

def show_function(fun, scale):
    '''Display image of function fun'''
    dim = 1024
    img = empty((dim,dim))
    s = (1/dim)
    for i,j in ndindex((dim,dim)):
        p = s*vec2(i,j)
        img[i,j] = fun(p, scale)
    fig = px.imshow(img, width=800, height=800)
    fig.show()


def show_height_map(fun, scale):
    '''Display function as a height map rendered as a 3D mesh.'''
    dim = 256
    img = ndarray((dim,dim))
    s = (scale/dim)
    for i,j in ndindex((dim,dim)):
        p = s*vec2(i,j)
        img[i,j] = fun(p)
    m = hmesh.Manifold()
    for i,j in ndindex((dim-1,dim-1)):
        p0 = vec3(i*s,j*s, img[i,j])
        p1 = vec3((i+1)*s,j*s, img[i+1,j])
        p2 = vec3((i+1)*s,(j+1)*s, img[i+1,j+1])
        p3 = vec3(i*s,(j+1)*s, img[i,j+1])
        m.add_face([p0, p1, p2, p3])
    hmesh.stitch(m)
    hmesh.triangulate(m)
    jd.display(m)


def show_PDS():
    t0 = time()
    pts,R = Poisson_Disk_Sampling()
    t1 = time()
    fig = px.scatter(x=pts[:,0], y=pts[:,1], width=800, height=800)
    fig.layout.yaxis.scaleanchor="x"
    fig.layout.xaxis.domain=[0.0, 1.0]
    fig.layout.yaxis.domain=[0.0, 1.0]
    fig.show()

def trace_curl_noise():
    global anim
    N = 256
    delta_x = cos(pi/3)
    delta_y = sin(pi/3)
    Ny = int(ceil(sqrt(N/delta_y)))
    Nx = int(Ny * delta_y)
    N = len(list(ndindex((Nx,Ny))))
    vec = zeros((100, N, 2))
    vec[0,:,:] = array([ vec2(i+(j%2)*delta_x,(j+0.5)*delta_y) / Nx for i,j in ndindex((Nx,Ny))])
    for t in range(1,100):
        for i in range(N):
            vec[t,i,:] = RK4(vec[t-1,i,:], dt=0.01, noise_scale=2)
    frames = repeat([i for i in range(100)], N)
    fig = px.scatter(x=vec[:,:,0].flatten(), y=vec[:,:,1].flatten(), width=800, height=800, animation_frame=frames)
    fig.layout.xaxis.domain=[0.0,1.0]
    fig.layout.yaxis.domain=[0.0,1.0]
    fig.show()

# Test your code using the functions below

In [ ]:
# Having solved Part 1 and 2 you should be able to generate
# noise images. The code below tests your implementation by
# showing the noise and its spectrum at two different scales
# Feel free to experiment with the scaling. 
#---
show_function_and_spectrum(noise, 3)
show_function_and_spectrum(noise, 30)

In [ ]:
# Having solved Part 3, you can generate a turbulence height map
# Test it using the function below. 
#---
show_height_map(turbulence, 0.5)


In [ ]:
# Having solved Part 4, you can use the function below to show your
# periodic pattern. 
#---
show_function(periodic_fun, 50)


In [ ]:
# Having solved Part 5, you can trace particles using curl noise. 
#---
trace_curl_noise()


In [ ]:
# Having solved Part 6, you can show a Poisson disk sampling using
# the code below. 
show_PDS()

# Questions
- Discuss the extent to which the noise function fulfills the desirable properties of noise (bandlimited, stationary, isotropic)
- Regarding turbulence, explain why the scaling of the noise octaves must decrease as the frequency goes up.
- When advecting particles using curl noise, explain why the particles don't bunch up or disperse over time.
- Explain why the Poisson disk sampling becomes extremely slow when the maximum number of points that the domain can accommodate is approached.
- Explain how curl noise can be used to generate the same type of point pattern as Poisson Disk Sampling.

